In [13]:
#Set up stuff from notebook
import os
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
from jaxtyping import Float
from functools import partial
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
from transformers import AutoTokenizer

DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2

IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
print("works")

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh

import plotly.io as pio

pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Neel")

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

device = utils.get_device()

works
Using renderer: notebook_connected


In [14]:
def get_params(model, show_param = False):
    count = 0
    parameters = []
    names = []
    for name, param in model.named_parameters():
        if show_param == True:
            print(name, param.size())
        names.append([name, param.size()])
        parameters.append(param.data)
        count += 1
    print(f"Model has {count} named parameters")
    return parameters, names


def unpack_names(names, first = 0, last = None):
    if last == None:
        for item in names[first:len(names)-1]:
            print(f"Param: {item[0]:<40} Dimensions of tensor: {item[1]:<30}")
    else:
        for item in names[first: last]:
            print(f"Param: {item[0]:<40} Dimensions of tensor: {str(item[1]):<30}")


def compare(model_1, model_2):
    different = []
    model_1_params, model_1_names = get_params(model_1)
    model_2_params, model_2_names = get_params(model_2)

    if len(model_1_params) != len(model_2_params):
        print("There are a different number of parameters between the models. Please check model config")
        print(f"Model 1 has {len(model_1_params)} parameters. Model 2 has {len(model_2_params)}")

    for i in range(min(len(model_1_params),(len(model_2_params)))):
        if model_1_names[i][1] != model_2_names[i][1]:
            different.append([model_1_names[i], model_2_names[i], ("index", i)])

    print(f"There are {len(different)} parameters that are different from each other")
    return different 

In [16]:
from transformers import GPTJForCausalLM        
model_path = r"C:\Users\allan\ResearchStuff\checkpoint-1953"

gptj = GPTJForCausalLM.from_pretrained(model_path)

gptj_params, gptj_names = get_params(gptj)

Model has 125 named parameters


In [17]:
config = {
    "d_model" : 768,
    "d_head" : 64,
    "n_layers" : 12,
    "n_ctx" : 1024,
    "n_heads" : 12,
    "d_mlp" : 3072, 
    "d_vocab" : 50257,
    "act_fn" : "gelu_new", 
    "eps" : 1e-05,
    "normalization_type" : "LN", 
    "positional_embedding_type" : "rotary",
    "rotary_dim": 64,
    "post_embedding_ln": False, 
    "original_architecture" : "gptj",
    "use_normalization_before_and_after" : False
}


hooked = HookedTransformer(config, tokenizer = AutoTokenizer.from_pretrained(model_path))
hooked_params, hooked_names = get_params(hooked)

Model has 197 named parameters


In [ ]:
def map_attn(model1 = hooked, model2 = gptj, num_blocks = 12): 
    for i in range(num_blocks):
        # Access state dict
        gptj_state_dict = gptj.state_dict()
        hooked_state_dict = hooked.state_dict()

        gptj_query_weight = f"transformer.h.{i}.attn.q_proj.weight"
        gptj_value_weight = f"transformer.h.{i}.attn.v_proj.weight"
        gptj_key_weight = f"transformer.h.{i}.attn.k_proj.weight"
        gptj_output_weight = f"transformer.h.{i}.attn.out_proj.weight"

        hooked_query_weight = f"blocks.{i}.attn.W_Q"
        hooked_value_weight = f"blocks.{i}.attn.W_V"
        hooked_key_weight = f"blocks.{i}.attn.W_K"
        hooked_output_weight = f"blocks.{i}.attn.W_O"

        # Extract weights
        query_weight_tensor = gptj_state_dict[gptj_query_weight]
        value_weight_tensor = gptj_state_dict[gptj_value_weight]
        key_weight_tensor = gptj_state_dict[gptj_key_weight]
        output_weight_tensor = gptj_state_dict[gptj_output_weight]

        # Reshaping the query, key, and value weights
        new_shape_query = query_weight_tensor.size()[:-1] + (12, 64) 
        query_weight_tensor = query_weight_tensor.view(new_shape_query)

        new_shape_value = value_weight_tensor.size()[:-1] + (12, 64)  
        value_weight_tensor = value_weight_tensor.view(new_shape_value)

        new_shape_key = key_weight_tensor.size()[:-1] + (12, 64)  
        key_weight_tensor = key_weight_tensor.view(new_shape_key)

        # Output reshaping:
        new_shape_output = (12, 64, 768)  # [768, 768] -> [12, 64, 768]
        output_weight_tensor = output_weight_tensor.view(new_shape_output)

        # Loop through each head 
        for j in range(12):  
            hooked_state_dict[hooked_query_weight][j,:,:] = query_weight_tensor[:,j,:]
            hooked_state_dict[hooked_value_weight][j,:,:] = value_weight_tensor[:,j,:]
            hooked_state_dict[hooked_key_weight][j,:,:] = key_weight_tensor[:,j,:]

            # Output has slightly different dimensions
            hooked_state_dict[hooked_output_weight][j,:,:] = output_weight_tensor[j,:,:]

        hooked.load_state_dict(hooked_state_dict)

        print(f"Successfully mapped attention weights for block {i} from GPT-J to Hooked Transformer.")


def set_bias_zero(model1 = hooked, model2 = gptj, num_blocks = 12):
    hooked_state_dict = hooked.state_dict()

    for i in range(12):
        hooked_query_bias = f"blocks.{i}.attn.b_Q"
        hooked_value_bias = f"blocks.{i}.attn.b_V"
        hooked_key_bias = f"blocks.{i}.attn.b_K"
        hooked_output_bias = f"blocks.{i}.attn.b_O"

        hooked_state_dict[hooked_query_bias][i, :] = torch.zeros_like(hooked_state_dict[hooked_query_bias][i, :])
        hooked_state_dict[hooked_value_bias][i, :] = torch.zeros_like(hooked_state_dict[hooked_value_bias][i, :])
        hooked_state_dict[hooked_key_bias][i, :] = torch.zeros_like(hooked_state_dict[hooked_key_bias][i, :])
        hooked_state_dict[hooked_output_bias][:] = torch.zeros_like(hooked_state_dict[hooked_output_bias])

    hooked.load_state_dict(hooked_state_dict)

def map_mlp(model1 = hooked, model2 = gptj, num_blocks = 12):
    for i in range(num_blocks):
        gptj_state_dict = gptj.state_dict()
        hooked_state_dict = hooked.state_dict()

        gptj_mlp_fc_in_weight = f"transformer.h.{i}.mlp.fc_in.weight"
        gptj_mlp_fc_in_bias = f"transformer.h.{i}.mlp.fc_in.bias"
        gptj_mlp_fc_out_weight = f"transformer.h.{i}.mlp.fc_out.weight"
        gptj_mlp_fc_out_bias = f"transformer.h.{i}.mlp.fc_out.bias"

        hooked_mlp_W_in = f"blocks.{i}.mlp.W_in"
        hooked_mlp_b_in = f"blocks.{i}.mlp.b_in"
        hooked_mlp_W_out = f"blocks.{i}.mlp.W_out"
        hooked_mlp_b_out = f"blocks.{i}.mlp.b_out"

        mlp_W_in_tensor = gptj_state_dict[gptj_mlp_fc_in_weight]
        mlp_b_in_tensor = gptj_state_dict[gptj_mlp_fc_in_bias]
        mlp_W_out_tensor = gptj_state_dict[gptj_mlp_fc_out_weight]
        mlp_b_out_tensor = gptj_state_dict[gptj_mlp_fc_out_bias]

        mlp_W_in_tensor = mlp_W_in_tensor.T  
        mlp_W_out_tensor = mlp_W_out_tensor.T 

        hooked_state_dict[hooked_mlp_W_in] = mlp_W_in_tensor
        hooked_state_dict[hooked_mlp_b_in] = mlp_b_in_tensor
        hooked_state_dict[hooked_mlp_W_out] = mlp_W_out_tensor
        hooked_state_dict[hooked_mlp_b_out] = mlp_b_out_tensor
        
        hooked.load_state_dict(hooked_state_dict)
        print(f"Successfully mapped MLP weights and biases for block {i} from GPT-J to Hooked Transformer.")

def map_embed(model1=hooked, model2=gptj):
    gptj_state_dict = gptj.state_dict()
    hooked_state_dict = hooked.state_dict()

    gptj_wte_weight = "transformer.wte.weight"  
    gptj_lm_head_bias = "lm_head.bias"         
    
    hooked_embed_W_E = "embed.W_E"             
    hooked_unembed_W_U = "unembed.W_U"         
    hooked_unembed_b_U = "unembed.b_U"         

    # Extract the weights and biases from GPT-J
    wte_weight_tensor = gptj_state_dict[gptj_wte_weight]  # Shape: [50257, 768]
    lm_head_bias_tensor = gptj_state_dict[gptj_lm_head_bias]  # Shape: [50257]
    
    hooked_state_dict[hooked_embed_W_E] = wte_weight_tensor  # [50257, 768] -> [50257, 768]
    hooked_state_dict[hooked_unembed_W_U] = wte_weight_tensor.T  # [50257, 768] -> [768, 50257]

    hooked_state_dict[hooked_unembed_b_U] = lm_head_bias_tensor  # [50257] -> [50257]

    hooked.load_state_dict(hooked_state_dict)
    print("Successfully mapped embedding/unembedding weights and biases from GPT-J to Hooked Transformer.")


def map_ln_params(model1 = hooked, model2 = gptj, num_blocks = 12): # Model 2's ln params will be transferred to Model 1
    for i in range(num_blocks):
        gptj_state_dict = gptj.state_dict()
        hooked_state_dict = hooked.state_dict()

        gptj_ln_1_w = f"transformer.h.{i}.ln_1.weight"
        gptj_ln_1_b = f"transformer.h.{i}.ln_1.bias"

        hooked_ln1_w = f"blocks.{i}.ln1.w"
        hooked_ln1_b = f"blocks.{i}.ln1.b"
        hooked_ln2_w = f"blocks.{i}.ln2.w"
        hooked_ln2_b = f"blocks.{i}.ln2.b"

        if gptj_ln_1_w in gptj_state_dict and gptj_ln_1_b in gptj_state_dict:
            # Extract the weight and bias from GPT-J
            ln1_w_tensor = gptj_state_dict[gptj_ln_1_w]
            ln1_b_tensor = gptj_state_dict[gptj_ln_1_b]

            # Now map them to the Hooked Transformer model
            if hooked_ln1_w in hooked_state_dict:
                print(f"Copying {gptj_ln_1_w} to {hooked_ln1_w}")
                hooked_state_dict[hooked_ln1_w] = ln1_w_tensor
            else:
                print(f"{hooked_ln1_w} not found in Hooked Transformer.")

            if hooked_ln1_b in hooked_state_dict:
                print(f"Copying {gptj_ln_1_b} to {hooked_ln1_b}")
                hooked_state_dict[hooked_ln1_b] = ln1_b_tensor
            else:
                print(f"{hooked_ln1_b} not found in Hooked Transformer.")

            # Now copy the same weights to ln2.w and ln2.b in the Hooked Transformer
            if hooked_ln2_w in hooked_state_dict:
                print(f"Copying {gptj_ln_1_w} to {hooked_ln2_w}")
                hooked_state_dict[hooked_ln2_w] = ln1_w_tensor  # Same weight for ln2
            else:
                print(f"{hooked_ln2_w} not found in Hooked Transformer.")

            if hooked_ln2_b in hooked_state_dict:
                print(f"Copying {gptj_ln_1_b} to {hooked_ln2_b}")
                hooked_state_dict[hooked_ln2_b] = ln1_b_tensor  # Same bias for ln2
            else:
                print(f"{hooked_ln2_b} not found in Hooked Transformer.")
        else:
            print(f"Missing {gptj_ln_1_w} or {gptj_ln_1_b} in GPT-J state dict.")

        hooked.load_state_dict(hooked_state_dict)
        print(f"Successfully updated Hooked Transformer with GPT-J layer normalization parameters for block {i}.")

    

In [19]:
def swap_param():
    #Swaps hooked model's linear nomalization parameters with custom
    map_ln_params()

    # Sets bias of Q, V, K, O to 0 
    set_bias_zero()
    
    #Swaps hooked model's Q, V, K, O weights with custom 
    map_attn()

    #Swaps hooked model's MLP weights+biases with custom 
    map_mlp()

    # Swaps embedding, unembed 
    map_embed()

In [21]:
model_description_text = """## Loading Models

HookedTransformer comes loaded with >40 open source GPT-style models. You can load any of them in with `HookedTransformer.from_pretrained(MODEL_NAME)`. See my explainer for documentation of all supported models, and this table for hyper-parameters and the name used to load them. Each model is loaded into the consistent HookedTransformer architecture, designed to be clean, consistent and interpretability-friendly.

For this demo notebook we'll look at GPT-2 Small, an 80M parameter model. To try the model the model out, let's find the loss on this paragraph!"""
loss = hooked(model_description_text, return_type="loss")
print("Model loss:", loss)

Model loss: tensor(11.4938, device='cuda:0')
